In [31]:
import openai
import os
openai.api_key = os.genenv("OPENAI_API_TOKEN")

MODEL = "gpt-3.5-turbo"


def get_completion(promt, model=MODEL):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a NER model. You are given a text and you have to find the entities in it."},
            {"role": "user", "content": promt}
        ],
        temperature=0,
    )

    return response.choices[0]['message']['content']

In [32]:
LABELS = ["COURT",  "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER", "DATE", "ORG", "GPE", "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS", "OTHER_PERSON"]

In [36]:
import json

with open("../../data/NER_TRAIN_JUDGEMENT.json", 'r') as f:
  train_data = json.load(f)

with open("../../data/NER_DEV/NER_DEV_JUDGEMENT.json", 'r') as f:
  valid_data = json.load(f)

In [37]:
import pandas as pd

df = pd.DataFrame(train_data).set_index("id")
df.head()

,annotations,data,meta
id,,,
90d9a97c7b7749ec8a4f460fda6f937e,"[{'result': [{'value': {'start': 90, 'end': 10...",{'text': ' (7) On specific query by the Bench...,{'source': 'tax_districtcourts judgement https...
a325c57ba5b84c6fa46bee65e6616633,"[{'result': [{'value': {'start': 26, 'end': 30...",{'text': 'He was also asked whether Agya <span...,{'source': 'criminal_punjab-haryana_high_court...
33cface31108441080976035ee04d07e,"[{'result': [{'value': {'start': 13, 'end': 25...","{'text': ' 5.2 CW3 Mr Vijay Mishra , Deputy M...",{'source': 'financial_districtcourts judgement...
08debe900b51464e9f26b5d1eecae170,[{'result': []}],{'text': 'You are hereby asked not to carry ou...,{'source': 'civil_bombay_high_court judgement ...
428070222cd940eba78b233829eb28b8,"[{'result': [{'value': {'start': 18, 'end': 43...",{'text': 'The pillion rider T.V. Satyanarayana...,{'source': 'motorvehicles_andhra_high_court ju...


In [39]:
for i in [10,25,61]:
    print(df['data'][i]['text'])
    for j in range(len(df['annotations'][i][0]['result'])):
        print(df['annotations'][i][0]['result'][j]['value'])
    print()

On the night of 28 March, 1959, Krishnamurthi Rao, according to his story, walked into the room of the petitioner, gave the bribe and made the pre-arranged signal which brought the Assistant Superintendent of Police and his companions to the room.
{'start': 16, 'end': 30, 'text': '28 March, 1959', 'labels': ['DATE']}
{'start': 32, 'end': 49, 'text': 'Krishnamurthi Rao', 'labels': ['OTHER_PERSON']}

A contention similar to the one now raised by Mr. Dasgupta was also raised before the Supreme Court.
{'start': 50, 'end': 58, 'text': 'Dasgupta', 'labels': ['OTHER_PERSON']}
{'start': 86, 'end': 99, 'text': 'Supreme Court', 'labels': ['COURT']}

Dy.Registrar was competent to pass winding up order as powers were delegated to him vide notification dated 18.10.77 issued by Delhi Administration, Delhi.
{'start': 108, 'end': 116, 'text': '18.10.77', 'labels': ['DATE']}
{'start': 127, 'end': 147, 'text': 'Delhi Administration', 'labels': ['ORG']}
{'start': 149, 'end': 154, 'text': 'Delhi', 'labels

In [33]:
text = f"""
Treat the following text as a legal document and find the entities in it. \
You only have following entities to find: {", ".join(LABELS)}. \
Do not show any other entities. \
Remember to correctly show the place of the entity in the text. Do not show the same entity twice.  \
Do not confuse the start and end of the entity. \
Use the same format as in the example below. \
If you are not sure about the entity, leave it blank. \
"""

promt = """\n
INPUT: {'text': "\n\n(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy."}

OUTPUT: {'start': 90, 'end': 103, 'text': 'Hongkong Bank', 'labels': ['ORG']}
{'start': 267, 'end': 278, 'text': 'Rahul & Co.', 'labels': ['ORG']}

INPUT: {'text': 'He was also asked whether Agya <span class="hidden_text" id="span_5"> CRA No.326-DB of 1998 6</span> Kaur, mother-in-law of the deceased lived separately from Tarlochan Singh.'}

OUTPUT: {'start': 26, 'end': 30, 'text': 'Agya', 'labels': ['OTHER_PERSON']}
{'start': 101, 'end': 105, 'text': 'Kaur', 'labels': ['OTHER_PERSON']}
{'start': 159, 'end': 174, 'text': 'Tarlochan Singh', 'labels': ['OTHER_PERSON']}


INPUT: {'text': ' \n5.2 CW3 Mr Vijay Mishra , Deputy Manager, HDFC Bank, Noida, UP has deposed that complainant had a current account with HDFC Bank in the year 2004\xad2005.'}

OUTPUT: {'start': 13, 'end': 25, 'text': 'Vijay Mishra', 'labels': ['WITNESS']}
{'start': 44, 'end': 60, 'text': 'HDFC Bank, Noida', 'labels': ['ORG']}
{'start': 62, 'end': 64, 'text': 'UP', 'labels': ['GPE']}
{'start': 121, 'end': 130, 'text': 'HDFC Bank', 'labels': ['ORG']}

INPUT: {'text': 'On the night of 28 March, 1959, Krishnamurthi Rao, according to his story, walked into the room of the petitioner, gave the bribe and made the pre-arranged signal which brought the Assistant Superintendent of Police and his companions to the room.'}

OUTPUT:
"""

output = get_completion(text + promt)
print(output)

{'start': 47, 'end': 63, 'text': 'Krishnamurthi Rao', 'labels': ['OTHER_PERSON']}
{'start': 87, 'end': 97, 'text': 'petitioner', 'labels': ['PETITIONER']}
{'start': 123, 'end': 152, 'text': 'Assistant Superintendent of Police', 'labels': ['OTHER_PERSON']}


In [59]:
examples_per_label = {label: [] for label in LABELS}
for i in range(len(valid_data)):
    for j in range(len(valid_data[i]['annotations'][0]['result'])):
        label = valid_data[i]['annotations'][0]['result'][j]['value']['labels'][0]
        examples_per_label[label].append(valid_data[i])

#keep 2 examples per label
for label in examples_per_label:
    examples_per_label[label] = examples_per_label[label][:1]

examples_per_label['GPE'][0]['annotations'][0]['result']

[{'value': {'start': 101, 'end': 108, 'text': 'England', 'labels': ['GPE']},
  'id': 'Q3J9O1UZ',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'},
 {'value': {'start': 134,
   'end': 140,
   'text': 'Heydon',
   'labels': ['OTHER_PERSON']},
  'id': 'ETZMBDT6',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'}]

In [61]:
# create promt where I show 2 examples per label as few shot learning
promt = ""
for label in examples_per_label:
    for example in examples_per_label[label]:
        promt += f"\n\nINPUT: {example['data']}\n\nOUTPUT:"
        for result in example['annotations'][0]['result']:
            promt += f" {result['value']}"

promt 


'\n\nINPUT: {\'text\': \'The LoA stipulated that:\\n\\n "17. The allotment of said land is subject to the outcome of the Appeal Suit No. 274/2007 in (OS No. 155/05), WP Nos. 19670/07, 20667/07 and 22043/07 pending before the Hon\\\'ble High Court of Andhra Pradesh." \\n\\n PART A 4\'}\n\nOUTPUT: {\'start\': 92, \'end\': 116, \'text\': \'Appeal Suit No. 274/2007\', \'labels\': [\'CASE_NUMBER\']} {\'start\': 121, \'end\': 134, \'text\': \'OS No. 155/05\', \'labels\': [\'CASE_NUMBER\']} {\'start\': 137, \'end\': 176, \'text\': \'WP Nos. 19670/07, 20667/07 and 22043/07\', \'labels\': [\'CASE_NUMBER\']} {\'start\': 204, \'end\': 232, \'text\': \'High Court of Andhra Pradesh\', \'labels\': [\'COURT\']}\n\nINPUT: {\'text\': "The petitioner Sukhbir Kataria, who is accused no.14 in Criminal Complaint bearing No.74136 dated 04.06.2013 (Annexure P-1) titled as \'Om Parkash Vs. Satish Kumar and others\' seeks quashing of the same alongwith the summoning order dated 22.07.2013 (Annexure P-2), where

In [62]:
#get random example from valid data which is not in examples_per_label
import random

for i in range(100):
    random_example = random.choice(valid_data)
    if random_example not in examples_per_label[random_example['annotations'][0]['result'][0]['value']['labels'][0]]:
        break

random_example

{'id': 'e8ab02df67d54b398e53984e751348dc',
 'annotations': [{'result': [{'value': {'start': 71,
      'end': 81,
      'text': 'Kiran Suri',
      'labels': ['OTHER_PERSON']},
     'id': 'I5ZUBW7K',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 114,
      'end': 131,
      'text': 'Bombay High Court',
      'labels': ['COURT']},
     'id': 'P3ZIFVVF',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 135,
      'end': 212,
      'text': 'Municipal Corporation of Greater Bombay v. Kisan Gangaram Hire [1987 ACJ 311]',
      'labels': ['PRECEDENT']},
     'id': 'B2IJBQZS',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels'}]}],
 'data': {'text': 'In support of her contention, the learned counsel for the appellant Ms.Kiran Suri referred to the decision of the Bombay High Court in Municipal Corporation of Greater Bombay v. Kisan Gangaram Hire [1987 ACJ 311] wherein the

In [63]:
promt += f"\n\nINPUT: {random_example['data']}\n\nOUTPUT:"


In [67]:
random_example['data']['text']

'In support of her contention, the learned counsel for the appellant Ms.Kiran Suri referred to the decision of the Bombay High Court in Municipal Corporation of Greater Bombay v. Kisan Gangaram Hire [1987 ACJ 311] wherein the Court dealt with a similar contention and observed thus:\n "8.'

In [64]:
output = get_completion(text + promt)
print(output)

{'start': 47, 'end': 62, 'text': 'Kiran Suri', 'labels': ['LAWYER']} {'start': 87, 'end': 103, 'text': 'Bombay High Court', 'labels': ['COURT']} {'start': 107, 'end': 139, 'text': 'Municipal Corporation of Greater Bombay', 'labels': ['ORG']} {'start': 143, 'end': 168, 'text': 'Kisan Gangaram Hire [1987 ACJ 311]', 'labels': ['PRECEDENT']}


In [65]:
random_example['annotations'][0]['result']

[{'value': {'start': 71,
   'end': 81,
   'text': 'Kiran Suri',
   'labels': ['OTHER_PERSON']},
  'id': 'I5ZUBW7K',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'},
 {'value': {'start': 114,
   'end': 131,
   'text': 'Bombay High Court',
   'labels': ['COURT']},
  'id': 'P3ZIFVVF',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'},
 {'value': {'start': 135,
   'end': 212,
   'text': 'Municipal Corporation of Greater Bombay v. Kisan Gangaram Hire [1987 ACJ 311]',
   'labels': ['PRECEDENT']},
  'id': 'B2IJBQZS',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'}]